# 2d

In [2]:
from hyper_tuning_helper_functions import (
    model_transference_2_classes_combined_method,
    fit_then_evaluate_model
)
from load_datasets import create_generators

## Transference combined model method

In [3]:
from tensorflow.keras.applications import Xception

In [8]:
# TODO 2: Feed that input to a mlp model and train on that data

image_size = 299

Xception_model = Xception(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

train_generator_299x299, val_generator_299x299 = create_generators((image_size, image_size))

# Xception_model.summary()

In [9]:
X_train_feature_maps = Xception_model.predict(train_generator_299x299)

2022-05-11 09:31:26.201064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [21]:
test_predict_generator = Xception_model.predict(train_generator_299x299)

In [22]:
test_predict_generator.shape

(1600, 10, 10, 2048)

In [16]:
import numpy as np

def extract_X_y_from_generator(generator):
    steps = generator.n//train_generator_299x299.batch_size
    X, y = [] , []
    for _ in range(steps):
        X_batch, y_batch = generator.next()
        X.extend(X_batch)
        y.extend(y_batch)
    return np.array(X), np.array(y)

In [26]:
def extract_feature_maps(model, generator):
    X, y = extract_X_y_from_generator(generator)
    return model.predict(X), y

X_train, y_train = extract_feature_maps(Xception_model, train_generator_299x299)

In [27]:
X_train.shape

(1600, 10, 10, 2048)